In [1]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
from collections import Counter
from typing import List, Dict, Tuple, Optional
import json

# NLP libraries
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import hdbscan  # Для кластеризации без фиксированного числа кластеров
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


/Users/a.danyarov/Library/Caches/pypoetry/virtualenvs/lamoda-bootcamp-9UBAvNEw-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ============================================================================
# PROJECT SETUP & MODEL INITIALIZATION
# ============================================================================

PROJECT_DIR = Path().resolve()
INTERIM_DIR = PROJECT_DIR / "data/interim"
FILE_PATH = PROJECT_DIR / "data/raw/lamoda_reviews.csv"

# Инициализация модели для embeddings
print("="*80)
print("INITIALIZING EMBEDDING MODEL")
print("="*80)

# Пробуем загрузить модель из кеша
model_candidates = [
    'paraphrase-multilingual-MiniLM-L12-v2',  # Лучше для русского
    'all-MiniLM-L6-v2',
    'all-mpnet-base-v2',
]

sentence_model = None
USE_TFIDF_FALLBACK = False

for model_name in model_candidates:
    try:
        print(f"Trying to load model: {model_name}...")
        sentence_model = SentenceTransformer(model_name, local_files_only=True)
        print(f"✓ Model '{model_name}' loaded successfully!")
        break
    except Exception as e:
        print(f"✗ {model_name}: Not found in cache")
        continue

if sentence_model is None:
    print("\n⚠ No sentence transformer models found in cache")
    print("Using TF-IDF as fallback for embeddings")
    USE_TFIDF_FALLBACK = True
else:
    print(f"\n✓ Using model: {model_name}")
    print(f"  Embedding dimension: {sentence_model.get_sentence_embedding_dimension()}")
print("="*80)


No sentence-transformers model found with name sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2. Creating a new one with mean pooling.
No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.
No sentence-transformers model found with name sentence-transformers/all-mpnet-base-v2. Creating a new one with mean pooling.


INITIALIZING EMBEDDING MODEL
Trying to load model: paraphrase-multilingual-MiniLM-L12-v2...
✗ paraphrase-multilingual-MiniLM-L12-v2: Not found in cache
Trying to load model: all-MiniLM-L6-v2...
✗ all-MiniLM-L6-v2: Not found in cache
Trying to load model: all-mpnet-base-v2...
✗ all-mpnet-base-v2: Not found in cache

⚠ No sentence transformer models found in cache
Using TF-IDF as fallback for embeddings


In [3]:
# ============================================================================
# ШАГ 0: ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ПО ТИПУ ПРОДУКТА
# ============================================================================
# Объекты: product_type, category, reviews (объединенные по типу продукта)

print("="*80)
print("ШАГ 0: ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ПО ТИПУ ПРОДУКТА")
print("="*80)

# Загружаем данные
print("Loading data...")
df_raw = pd.read_csv(FILE_PATH)
print(f"Loaded {len(df_raw)} reviews")
print(f"Unique products: {df_raw['product_sku'].nunique()}")
print(f"Unique product types: {df_raw['good_subtype'].nunique()}")
print(f"Categories: {df_raw['good_type'].unique()}")

# Группируем по ТИПУ ПРОДУКТА (product_type) вместо по продукту
print("\nPreparing data structure by product type...")
product_types_data = []

for product_type, group in df_raw.groupby('good_subtype'):
    if pd.isna(product_type):
        continue
    
    # Собираем все отзывы для всех продуктов этого типа
    reviews = group['comment_text'].dropna().tolist()
    
    if len(reviews) < 10:  # Минимум 10 отзывов для анализа типа продукта
        continue
    
    # Получаем категорию (берем самую частую)
    category = group['good_type'].mode()[0] if 'good_type' in group.columns else None
    
    # Собираем информацию о продуктах этого типа
    unique_skus = group['product_sku'].nunique()
    unique_names = group['name'].nunique()
    
    product_types_data.append({
        'product_type': product_type,
        'category': category,
        'reviews': reviews,
        'num_reviews': len(reviews),
        'num_products': unique_skus,
        'num_unique_names': unique_names
    })

# Создаем DataFrame
df_product_types = pd.DataFrame(product_types_data)
print(f"✓ Prepared {len(df_product_types)} product types with at least 10 reviews")

# Для тестирования можно использовать подмножество
USE_SUBSET = True
if USE_SUBSET:
    # Берем топ-20 типов продуктов по количеству отзывов
    df_product_types = df_product_types.nlargest(20, 'num_reviews')
    print(f"Using subset: {len(df_product_types)} product types for testing")

print(f"\nExample product type structure:")
example = df_product_types.iloc[0]
print(f"  Product Type: {example['product_type']}")
print(f"  Category: {example['category']}")
print(f"  Reviews: {example['num_reviews']}")
print(f"  Products: {example['num_products']}")
print(f"  Sample review: {example['reviews'][0][:80]}...")

df_product_types.head()


ШАГ 0: ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ПО ТИПУ ПРОДУКТА
Loading data...
Loaded 1774267 reviews
Unique products: 254307
Unique product types: 119
Categories: ['Clothes' 'Accs' 'Shoes' 'Beauty_Accs' 'Home_Accs' 'Bags' 'Toys'
 'Jewellery' nan]

Preparing data structure by product type...
✓ Prepared 112 product types with at least 10 reviews
Using subset: 20 product types for testing

Example product type structure:
  Product Type: TEE-SHIRTS & POLOS
  Category: Clothes
  Reviews: 170470
  Products: 23469
  Sample review: Произведено в Турции. Качество хорошее и размеру соответствует....


,product_type,category,reviews,num_reviews,num_products,num_unique_names
99,TEE-SHIRTS & POLOS,Clothes,[Произведено в Турции. Качество хорошее и разм...,170470,23469,56
90,SPORT SHOES,Shoes,"[Мужу понравились. , Кеды как кеды.купила за 1...",162149,20612,32
105,TROUSERS & JUMPSUITS,Clothes,"[Ничего плохого в них не вижу , Хорошие штаниш...",124705,18374,49
73,OUTWEAR,Clothes,"[Купили сыну на день рождения!Теплая, но не жа...",112425,18712,94
108,UNDERWEAR,Clothes,"[трусы огонь, Хорошие трусы, Хорошо все, Очень...",90187,11697,105


In [4]:
# ============================================================================
# ШАГ 1: ОЧИСТКА ТЕКСТОВ
# ============================================================================
# Вход: reviews: List[str]
# Выход: clean_reviews: List[str]

print("="*80)
print("ШАГ 1: ОЧИСТКА ТЕКСТОВ")
print("="*80)

def clean_text_soft(text: str) -> str:
    """
    Мягкая очистка текста (feature cleaning, не feature engineering):
    - Приводим к нижнему регистру
    - Убираем HTML теги
    - Убираем лишние символы
    - НЕ удаляем слова по смыслу
    """
    if pd.isna(text) or not text:
        return ""
    
    text = str(text)
    
    # Убираем HTML теги
    text = re.sub(r'<[^>]+>', '', text)
    
    # Убираем множественные пробелы и переносы строк
    text = re.sub(r'\s+', ' ', text)
    
    # Убираем управляющие символы
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    
    # Приводим к нижнему регистру
    text = text.strip().lower()
    
    return text


# Применяем очистку ко всем отзывам всех типов продуктов
print("Cleaning reviews...")
df_product_types['clean_reviews'] = df_product_types['reviews'].apply(
    lambda reviews: [clean_text_soft(r) for r in reviews if clean_text_soft(r)]
)

# Удаляем типы продуктов без отзывов после очистки
df_product_types = df_product_types[df_product_types['clean_reviews'].apply(len) >= 10].copy()
df_product_types['num_clean_reviews'] = df_product_types['clean_reviews'].apply(len)

print(f"✓ Cleaned reviews for {len(df_product_types)} product types")
print(f"  Total reviews before: {df_product_types['num_reviews'].sum()}")
print(f"  Total reviews after: {df_product_types['num_clean_reviews'].sum()}")

# Показываем пример
example = df_product_types.iloc[0]
print(f"\nExample (Product Type: {example['product_type']}):")
print(f"  Original: {example['reviews'][0][:100]}...")
print(f"  Cleaned:  {example['clean_reviews'][0][:100]}...")

df_product_types[['product_type', 'category', 'num_clean_reviews', 'num_products']].head()


ШАГ 1: ОЧИСТКА ТЕКСТОВ
Cleaning reviews...
✓ Cleaned reviews for 20 product types
  Total reviews before: 1396425
  Total reviews after: 1396405

Example (Product Type: TEE-SHIRTS & POLOS):
  Original: Произведено в Турции. Качество хорошее и размеру соответствует....
  Cleaned:  произведено в турции. качество хорошее и размеру соответствует....


,product_type,category,num_clean_reviews,num_products
99,TEE-SHIRTS & POLOS,Clothes,170470,23469
90,SPORT SHOES,Shoes,162145,20612
105,TROUSERS & JUMPSUITS,Clothes,124703,18374
73,OUTWEAR,Clothes,112421,18712
108,UNDERWEAR,Clothes,90185,11697


In [5]:
# ============================================================================
# ШАГ 2: ПРЕОБРАЗОВАНИЕ ОТЗЫВОВ В ЧИСЛОВЫЕ ВЕКТОРЫ (EMBEDDINGS)
# ============================================================================
# Вход: clean_reviews: List[str]
# Выход: review_embeddings: ndarray shape (n_reviews, embedding_dim)

print("="*80)
print("ШАГ 2: ГЕНЕРАЦИЯ EMBEDDINGS")
print("="*80)

def get_embeddings(texts: List[str], batch_size: int = 32) -> np.ndarray:
    """
    Преобразование текстов в числовые векторы фиксированной длины
    FeatureExtractor(text) → R^embedding_dim
    """
    if USE_TFIDF_FALLBACK:
        print(f"Using TF-IDF embeddings (fallback mode)...")
        vectorizer = TfidfVectorizer(
            max_features=384,
            ngram_range=(1, 2),
            min_df=1,
            max_df=0.95
        )
        embeddings = vectorizer.fit_transform(texts).toarray()
        print(f"✓ Generated TF-IDF embeddings shape: {embeddings.shape}")
        return embeddings
    else:
        if sentence_model is None:
            raise ValueError("Sentence transformer model is not loaded!")
        
        print(f"Generating sentence transformer embeddings...")
        embeddings = sentence_model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=True,
            convert_to_numpy=True
        )
        print(f"✓ Generated embeddings shape: {embeddings.shape}")
        return embeddings


# Генерируем embeddings для каждого типа продукта
print("Processing product types...")
all_embeddings = []

for idx, row in df_product_types.iterrows():
    clean_reviews = row['clean_reviews']
    
    # Получаем embeddings для всех отзывов типа продукта
    embeddings = get_embeddings(clean_reviews)
    all_embeddings.append(embeddings)

# Сохраняем embeddings в DataFrame
df_product_types['review_embeddings'] = all_embeddings

print(f"\n✓ Generated embeddings for {len(df_product_types)} product types")
print(f"  Embedding dimension: {df_product_types['review_embeddings'].iloc[0].shape[1]}")

# Показываем пример
example = df_product_types.iloc[0]
print(f"\nExample (Product Type: {example['product_type']}):")
print(f"  Reviews: {len(example['clean_reviews'])}")
print(f"  Embeddings shape: {example['review_embeddings'].shape}")

df_product_types[['product_type', 'category', 'num_clean_reviews']].head()


ШАГ 2: ГЕНЕРАЦИЯ EMBEDDINGS
Processing product types...
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (170470, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (162145, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (124703, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (112421, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (90185, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (89737, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (87520, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (80475, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (73908, 384)
Using TF-IDF embeddings (fallback mode)...
✓ Generated TF-IDF embeddings shape: (52810, 384)
Using TF-I

,product_type,category,num_clean_reviews
99,TEE-SHIRTS & POLOS,Clothes,170470
90,SPORT SHOES,Shoes,162145
105,TROUSERS & JUMPSUITS,Clothes,124703
73,OUTWEAR,Clothes,112421
108,UNDERWEAR,Clothes,90185


In [1]:
# ============================================================================
# ШАГ 3: КЛАСТЕРИЗАЦИЯ ОТЗЫВОВ ПО СМЫСЛУ
# ============================================================================
# Вход: review_embeddings: ndarray
# Выход: clusters: List[int]  # cluster_id для каждого отзыва
# Применяем кластеризацию БЕЗ заранее заданного числа кластеров
# Цель: сгруппировать отзывы по тому, о чём они (материал, комфорт, доставка, качество)

print("="*80)
print("ШАГ 3: КЛАСТЕРИЗАЦИЯ ОТЗЫВОВ")
print("="*80)

def cluster_reviews(embeddings: np.ndarray, min_cluster_size: int = 5) -> np.ndarray:
    """
    Кластеризация отзывов БЕЗ фиксированного числа кластеров
    Используем HDBSCAN для автоматического определения числа кластеров
    Возвращает массив cluster_id для каждого отзыва (-1 = шум/outlier)
    """
    n_samples = len(embeddings)
    
    if n_samples < min_cluster_size:
        # Если отзывов слишком мало, все в один кластер
        return np.zeros(n_samples, dtype=int)
    
    # Используем HDBSCAN для автоматической кластеризации
    # min_cluster_size - минимальный размер кластера (больше для типов продуктов)
    # min_samples - минимальное количество соседей для core point
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=2,
        metric='euclidean',
        cluster_selection_method='eom'  # Excess of Mass
    )
    
    cluster_labels = clusterer.fit_predict(embeddings)
    
    # Если все отзывы стали шумом, используем KMeans как fallback
    if np.all(cluster_labels == -1):
        n_clusters = min(5, n_samples // 10)
        if n_clusters > 0:
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
            cluster_labels = kmeans.fit_predict(embeddings)
    
    return cluster_labels


# Применяем кластеризацию для каждого типа продукта
print("Clustering reviews for each product type...")
all_clusters = []

for idx, row in df_product_types.iterrows():
    embeddings = row['review_embeddings']
    clean_reviews = row['clean_reviews']
    
    # Кластеризуем отзывы типа продукта
    # Используем больший min_cluster_size, т.к. у нас больше данных
    cluster_labels = cluster_reviews(embeddings, min_cluster_size=5)
    
    all_clusters.append(cluster_labels)
    
    # Показываем статистику для первых нескольких типов продуктов
    if idx < 3:
        unique, counts = np.unique(cluster_labels, return_counts=True)
        print(f"\n  Product Type: {row['product_type']}")
        print(f"    Reviews: {len(clean_reviews)}")
        print(f"    Clusters: {len(unique[unique >= 0])} (noise: {np.sum(cluster_labels == -1)})")
        for cluster_id, count in zip(unique, counts):
            if cluster_id >= 0:
                print(f"      Cluster {cluster_id}: {count} reviews")

# Сохраняем кластеры
df_product_types['clusters'] = all_clusters

print(f"\n✓ Clustered reviews for {len(df_product_types)} product types")

# Показываем общую статистику
total_clusters = sum(len(np.unique(c[c >= 0])) for c in all_clusters)
total_noise = sum(np.sum(c == -1) for c in all_clusters)
print(f"  Total clusters found: {total_clusters}")
print(f"  Total noise/outliers: {total_noise}")

df_product_types[['product_type', 'category', 'num_clean_reviews']].head()


ШАГ 3: КЛАСТЕРИЗАЦИЯ ОТЗЫВОВ


NameError: name 'np' is not defined

In [ ]:
# ============================================================================
# ШАГ 4: РАБОТА С КЛАСТЕРАМИ (НЕ С ОТЗЫВАМИ)
# ============================================================================
# Объект меняется: теперь основная единица анализа = Cluster
# Cluster = набор отзывов с общим смыслом

print("="*80)
print("ШАГ 4: ГРУППИРОВКА ПО КЛАСТЕРАМ")
print("="*80)

def group_reviews_by_clusters(
    clean_reviews: List[str],
    clusters: np.ndarray,
    embeddings: np.ndarray
) -> Dict[int, Dict]:
    """
    Группировка отзывов по кластерам
    Возвращает словарь {cluster_id: {'texts': [...], 'embeddings': [...]}}
    """
    cluster_groups = {}
    
    for cluster_id in np.unique(clusters):
        if cluster_id == -1:
            # Пропускаем шум
            continue
        
        # Находим индексы отзывов в этом кластере
        cluster_indices = np.where(clusters == cluster_id)[0]
        
        cluster_groups[cluster_id] = {
            'texts': [clean_reviews[i] for i in cluster_indices],
            'embeddings': embeddings[cluster_indices]
        }
    
    return cluster_groups


# Группируем отзывы по кластерам для каждого типа продукта
print("Grouping reviews by clusters...")
all_cluster_groups = []

for idx, row in df_product_types.iterrows():
    clean_reviews = row['clean_reviews']
    clusters = row['clusters']
    embeddings = row['review_embeddings']
    
    # Группируем по кластерам
    cluster_groups = group_reviews_by_clusters(clean_reviews, clusters, embeddings)
    all_cluster_groups.append(cluster_groups)
    
    # Показываем примеры для первых типов продуктов
    if idx < 3:
        print(f"\n  Product Type: {row['product_type']} ({row['category']}):")
        for cluster_id, group in list(cluster_groups.items())[:3]:
            print(f"    Cluster {cluster_id}: {len(group['texts'])} reviews")
            print(f"      Example: {group['texts'][0][:80]}...")

# Сохраняем группировку
df_product_types['cluster_groups'] = all_cluster_groups

print(f"\n✓ Grouped reviews into clusters for {len(df_product_types)} product types")

# Статистика
total_clusters = sum(len(groups) for groups in all_cluster_groups)
avg_cluster_size = np.mean([
    len(texts) 
    for groups in all_cluster_groups 
    for cluster_id, group in groups.items() 
    for texts in [group['texts']]
])
print(f"  Total clusters: {total_clusters}")
print(f"  Average cluster size: {avg_cluster_size:.2f} reviews")

df_product_types[['product_type', 'category', 'num_clean_reviews']].head()


In [ ]:
# ============================================================================
# ШАГ 5: ИЗВЛЕЧЕНИЕ АСПЕКТОВ ИЗ КЛАСТЕРОВ
# ============================================================================
# Вход: texts_in_cluster: List[str]
# Выход: raw_aspects: List[str]
# Аспект = формулировка свойства товара, которое регулярно упоминается в кластере

print("="*80)
print("ШАГ 5: ИЗВЛЕЧЕНИЕ АСПЕКТОВ ИЗ КЛАСТЕРОВ")
print("="*80)

def extract_aspects_from_cluster(
    cluster_texts: List[str],
    min_word_freq: int = 3,
    max_aspects: int = 5
) -> List[str]:
    """
    Извлечение аспектов из кластера
    Находим top-features (слова/фразы), которые отличают этот кластер от других
    """
    if len(cluster_texts) == 0:
        return []
    
    # Объединяем все тексты кластера
    all_text = ' '.join(cluster_texts)
    
    # Разбиваем на слова
    words = re.findall(r'\b[а-яёa-z]{3,}\b', all_text.lower())
    
    if len(words) == 0:
        return []
    
    # Подсчитываем частоту слов
    word_counts = Counter(words)
    
    # Фильтруем слишком частые (стоп-слова) и слишком редкие
    filtered_words = {
        word: count 
        for word, count in word_counts.items() 
        if count >= min_word_freq and count < len(cluster_texts) * 0.9
    }
    
    # Сортируем по частоте и берем топ
    top_words = sorted(filtered_words.items(), key=lambda x: x[1], reverse=True)[:max_aspects]
    
    # Формируем аспекты из топ-слов
    aspects = []
    for word, count in top_words:
        # Ищем предложения, содержащие это слово
        for text in cluster_texts:
            if word in text.lower():
                # Берем короткую фразу вокруг слова
                words_in_text = text.split()
                if word in [w.lower() for w in words_in_text]:
                    # Берем до 5 слов вокруг ключевого слова
                    word_idx = [i for i, w in enumerate(words_in_text) if word in w.lower()]
                    if word_idx:
                        start = max(0, word_idx[0] - 2)
                        end = min(len(words_in_text), word_idx[0] + 3)
                        phrase = ' '.join(words_in_text[start:end])
                        if len(phrase) > 10 and phrase not in aspects:
                            aspects.append(phrase)
                            break
        
        if len(aspects) >= max_aspects:
            break
    
    # Если не нашли фразы, возвращаем просто топ-слова
    if len(aspects) == 0:
        aspects = [word for word, _ in top_words[:max_aspects]]
    
    return aspects[:max_aspects]


# Извлекаем аспекты для каждого кластера каждого типа продукта
print("Extracting aspects from clusters...")
all_product_type_aspects = []

for idx, row in df_product_types.iterrows():
    cluster_groups = row['cluster_groups']
    
    product_type_aspects = {}
    
    for cluster_id, group in cluster_groups.items():
        cluster_texts = group['texts']
        
        # Извлекаем аспекты из кластера
        aspects = extract_aspects_from_cluster(cluster_texts, min_word_freq=3, max_aspects=5)
        
        if aspects:
            product_type_aspects[cluster_id] = aspects
    
    all_product_type_aspects.append(product_type_aspects)
    
    # Показываем примеры для первых типов продуктов
    if idx < 3 and product_type_aspects:
        print(f"\n  Product Type: {row['product_type']} ({row['category']}):")
        for cluster_id, aspects in list(product_type_aspects.items())[:2]:
            print(f"    Cluster {cluster_id}: {aspects}")

# Сохраняем аспекты
df_product_types['raw_aspects'] = all_product_type_aspects

print(f"\n✓ Extracted aspects for {len(df_product_types)} product types")

# Статистика
total_aspects = sum(len(aspects) for aspects in all_product_type_aspects)
print(f"  Total aspects extracted: {total_aspects}")
print(f"  Average aspects per product type: {total_aspects / len(df_product_types):.2f}")

df_product_types[['product_type', 'category', 'raw_aspects']].head()


In [ ]:
# ============================================================================
# ШАГ 6: ФИЛЬТРАЦИЯ АСПЕКТОВ С УЧЕТОМ КАТЕГОРИИ
# ============================================================================
# Вход: raw_aspects + category
# Выход: filtered_aspects: List[str]

print("="*80)
print("ШАГ 6: ФИЛЬТРАЦИЯ АСПЕКТОВ (CATEGORY-AWARE)")
print("="*80)

# Словари запрещенных и приоритетных аспектов для разных категорий
FORBIDDEN_ASPECTS = {
    'Clothes': ['доставка', 'упаковка'],
    'Shoes': ['доставка', 'упаковка'],
    'Accs': ['доставка', 'упаковка'],
    'Beauty_Accs': ['размер'],
}

PRIORITY_ASPECTS = {
    'Clothes': ['размер', 'материал', 'качество', 'комфорт'],
    'Shoes': ['размер', 'комфорт', 'качество'],
    'Accs': ['качество', 'материал', 'внешний вид'],
    'Beauty_Accs': ['качество', 'эффект', 'аромат'],
}

def filter_aspects(
    raw_aspects: Dict[int, List[str]],
    category: Optional[str]
) -> List[str]:
    """
    Фильтрация и приоритизация аспектов с учетом категории
    """
    # Собираем все аспекты в один список
    all_aspects = []
    for cluster_id, aspects in raw_aspects.items():
        all_aspects.extend(aspects)
    
    if len(all_aspects) == 0:
        return []
    
    # Фильтруем запрещенные аспекты
    forbidden = FORBIDDEN_ASPECTS.get(category, [])
    filtered = [
        aspect for aspect in all_aspects
        if not any(forbidden_word in aspect.lower() for forbidden_word in forbidden)
    ]
    
    # Если после фильтрации ничего не осталось, возвращаем исходные
    if len(filtered) == 0:
        filtered = all_aspects
    
    # Приоритизируем важные аспекты
    priority = PRIORITY_ASPECTS.get(category, [])
    priority_aspects = []
    other_aspects = []
    
    for aspect in filtered:
        is_priority = any(priority_word in aspect.lower() for priority_word in priority)
        if is_priority:
            priority_aspects.append(aspect)
        else:
            other_aspects.append(aspect)
    
    # Сортируем: сначала приоритетные, потом остальные
    result = priority_aspects + other_aspects
    
    # Ограничиваем количество (максимум 8 аспектов для типа продукта)
    return result[:8]


# Применяем фильтрацию для каждого типа продукта
print("Filtering aspects by category...")
all_filtered_aspects = []

for idx, row in df_product_types.iterrows():
    raw_aspects = row['raw_aspects']
    category = row['category']
    
    # Фильтруем аспекты
    filtered = filter_aspects(raw_aspects, category)
    all_filtered_aspects.append(filtered)
    
    # Показываем примеры для первых типов продуктов
    if idx < 3:
        print(f"\n  Product Type: {row['product_type']} ({category}):")
        print(f"    Raw aspects: {sum(len(a) for a in raw_aspects.values())}")
        print(f"    Filtered aspects ({len(filtered)}): {filtered[:5]}...")

# Сохраняем отфильтрованные аспекты
df_product_types['filtered_aspects'] = all_filtered_aspects
df_product_types['num_aspects'] = [len(a) for a in all_filtered_aspects]

print(f"\n✓ Filtered aspects for {len(df_product_types)} product types")

# Статистика
total_aspects = sum(len(a) for a in all_filtered_aspects)
print(f"  Total filtered aspects: {total_aspects}")
print(f"  Average aspects per product type: {total_aspects / len(df_product_types):.2f}")

df_product_types[['product_type', 'category', 'num_aspects', 'filtered_aspects']].head()


In [ ]:
# ============================================================================
# РЕЗУЛЬТАТЫ И СТАТИСТИКА
# ============================================================================

print("="*80)
print("РЕЗУЛЬТАТЫ ПАЙПЛАЙНА")
print("="*80)

# Формируем финальный DataFrame с результатами
results_df = df_product_types[[
    'product_type', 'category', 
    'num_reviews', 'num_products', 'num_aspects', 'filtered_aspects'
]].copy()

print(f"\n✓ Final results for {len(results_df)} product types")
print(f"\nStatistics:")
print(f"  Product types with aspects: {(results_df['num_aspects'] > 0).sum()}")
print(f"  Average aspects per product type: {results_df['num_aspects'].mean():.2f}")
print(f"  Min aspects: {results_df['num_aspects'].min()}")
print(f"  Max aspects: {results_df['num_aspects'].max()}")
print(f"  Median aspects: {results_df['num_aspects'].median():.2f}")

# Дополнительная статистика
print(f"\n  Total reviews processed: {results_df['num_reviews'].sum():,}")
print(f"  Total products covered: {results_df['num_products'].sum():,}")
print(f"  Average reviews per product type: {results_df['num_reviews'].mean():.0f}")
print(f"  Average products per product type: {results_df['num_products'].mean():.1f}")

# Статистика по кластерам
print(f"\n  Clustering statistics:")
total_clusters = sum(len(groups) for groups in all_cluster_groups)
total_noise = sum(np.sum(c == -1) for c in all_clusters)
total_reviews_clustered = sum(len(c) for c in all_clusters)
print(f"    Total clusters: {total_clusters}")
print(f"    Average clusters per product type: {total_clusters / len(results_df):.2f}")
print(f"    Noise/outliers: {total_noise} ({100*total_noise/total_reviews_clustered:.1f}%)")

# Статистика по категориям
print(f"\n  Statistics by category:")
category_stats = results_df.groupby('category').agg({
    'num_aspects': ['count', 'mean', 'sum'],
    'num_reviews': 'sum',
    'num_products': 'sum'
}).round(2)
print(category_stats)

results_df.head(10)


In [ ]:
# ============================================================================
# ПРИМЕРЫ РЕЗУЛЬТАТОВ ПО КАТЕГОРИЯМ
# ============================================================================

print("="*80)
print("EXAMPLES BY CATEGORY")
print("="*80)

for category in results_df['category'].dropna().unique()[:3]:
    cat_types = results_df[results_df['category'] == category].head(3)
    print(f"\n{category}:")
    print("-" * 80)
    for _, row in cat_types.iterrows():
        print(f"\n  Product Type: {row['product_type']}")
        print(f"  Reviews: {row['num_reviews']:,}, Products: {row['num_products']}, Aspects: {row['num_aspects']}")
        if row['filtered_aspects']:
            print(f"  Aspects:")
            for i, aspect in enumerate(row['filtered_aspects'][:6], 1):
                print(f"    {i}. {aspect}")


In [ ]:
# ============================================================================
# ВИЗУАЛИЗАЦИЯ СТАТИСТИК
# ============================================================================

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Распределение количества аспектов
axes[0, 0].hist(results_df['num_aspects'], bins=15, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Number of Aspects')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of Aspects per Product Type')
axes[0, 0].grid(True, alpha=0.3)

# 2. Количество аспектов по категориям
category_aspects = results_df.groupby('category')['num_aspects'].mean().sort_values(ascending=False)
axes[0, 1].barh(category_aspects.index, category_aspects.values)
axes[0, 1].set_xlabel('Average Number of Aspects')
axes[0, 1].set_title('Average Aspects by Category')
axes[0, 1].grid(True, alpha=0.3, axis='x')

# 3. Распределение количества отзывов
axes[1, 0].hist(results_df['num_reviews'], bins=20, edgecolor='black', alpha=0.7)
axes[1, 0].set_xlabel('Number of Reviews')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Distribution of Reviews per Product Type')
axes[1, 0].set_yscale('log')
axes[1, 0].grid(True, alpha=0.3)

# 4. Соотношение отзывов и аспектов
axes[1, 1].scatter(results_df['num_reviews'], results_df['num_aspects'], alpha=0.6)
axes[1, 1].set_xlabel('Number of Reviews')
axes[1, 1].set_ylabel('Number of Aspects')
axes[1, 1].set_title('Reviews vs Aspects')
axes[1, 1].set_xscale('log')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Дополнительная статистика
print("\n" + "="*80)
print("ДЕТАЛЬНАЯ СТАТИСТИКА")
print("="*80)

print(f"\nРаспределение аспектов:")
print(results_df['num_aspects'].describe())

print(f"\nТоп-10 типов продуктов по количеству отзывов:")
top_reviews = results_df.nlargest(10, 'num_reviews')[['product_type', 'category', 'num_reviews', 'num_aspects']]
print(top_reviews.to_string(index=False))

print(f"\nТоп-10 типов продуктов по количеству аспектов:")
top_aspects = results_df.nlargest(10, 'num_aspects')[['product_type', 'category', 'num_reviews', 'num_aspects']]
print(top_aspects.to_string(index=False))


In [ ]:
# ============================================================================
# ЭКСПОРТ РЕЗУЛЬТАТОВ
# ============================================================================

print("="*80)
print("ЭКСПОРТ РЕЗУЛЬТАТОВ")
print("="*80)

# Подготавливаем данные для экспорта
export_df = results_df.copy()
export_df['aspects'] = export_df['filtered_aspects'].apply(
    lambda x: ' | '.join(x) if isinstance(x, list) else ''
)

# Сохраняем в CSV
output_csv = INTERIM_DIR / "product_type_aspects.csv"
export_df.to_csv(output_csv, index=False, encoding='utf-8')
print(f"✓ Results saved to CSV: {output_csv}")

# Сохраняем в JSON (с полными списками аспектов)
output_json = INTERIM_DIR / "product_type_aspects.json"
results_df.to_json(output_json, orient='records', force_ascii=False, indent=2)
print(f"✓ Results saved to JSON: {output_json}")

# Финальная статистика
print("\n" + "="*80)
print("ФИНАЛЬНАЯ СТАТИСТИКА")
print("="*80)
print(f"Total product types processed: {len(results_df)}")
print(f"Product types with aspects: {(results_df['num_aspects'] > 0).sum()}")
print(f"Average aspects per product type: {results_df['num_aspects'].mean():.2f}")
print(f"\nAspects distribution:")
print(results_df['num_aspects'].value_counts().sort_index())
print(f"\nCategories distribution:")
print(results_df['category'].value_counts())

print("\n" + "="*80)
print("ПАЙПЛАЙН ЗАВЕРШЕН")
print("="*80)
print("\nСтруктура пайплайна:")
print("  0. Исходные данные: product_type, category, reviews (объединенные по типу)")
print("  1. Очистка текстов (мягкие правила)")
print("  2. Преобразование в числовые векторы (embeddings)")
print("  3. Кластеризация отзывов по смыслу (без фиксированного числа кластеров)")
print("  4. Работа с кластерами (группировка отзывов)")
print("  5. Извлечение аспектов из кластера (поиск частых слов/фраз)")
print("  6. Фильтрация аспектов с учетом category")
print("\nРезультаты сохранены в:")
print(f"  - {output_csv}")
print(f"  - {output_json}")
